In [12]:
class Contact():
    def __init__(self, type: str, firstname: str, middlename: str, lastname: str, ssn: str, dob: str, gender: str, apply: str, smoke: str, pregnant: str, relationship: str, document: str):
        self.type = type
        self.firstname = firstname
        self.middlename = middlename
        self.lastname = lastname
        self.ssn = ssn
        self.dob = dob
        self.gender = gender
        self.apply = apply
        self.smoke = smoke
        self.pregnant = pregnant
        self.relationship = relationship
        self.document = document

documents = {
    'CIUDADANO': 'Passport',
    'ANTORCHA': 'Resident (I-551)',
    'RESIDENTE': 'Resident (I-551)',
    'ASILO POLITICO': 'Notice of Action (I-797)',
    'PASAPORTE DE VIAJERO': 'Refugee Travel (I-571)',
    'PERMISO DE TRABAJO': 'Employment Authorization (I-766)',
    'VISA': 'Visa (temporary I-551)',
    'NOTICIA DE ACCION': 'Notice of Action (I-797)',
    'I-94': 'Alien number or I-94 number',
}

relationships = {  
    'HIJO(A)': 'Child',
    'HIJASTRO(A)': 'Child-In-Law',
    'HERMANO(A)': 'Sibling',
    'PADRE/MADRE': 'Parent', 
    'TIO(A)': 'Pibling',
    'ABUELO(A)': 'Grandparent',
    'NIETO(A)': 'Grandchild',
    'SOBRINO(A)': 'Nibling', 
    'PRIMO(A)': 'Cousin',
    'SUEGRO(A)': 'Parent-In-Law',
}

In [13]:
import requests
import json

class ERPSync:
    
    def __init__(self, base_url, api_key, api_secret):
        self.base_url = base_url
        self.api_key = api_key
        self.api_secret = api_secret
        self.headers = {
            "Authorization": f"token {api_key}:{api_secret}"
        }
        
        endpoint = '/api/method/frappe.auth.get_logged_user'
        
        response = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)
        
        if response.status_code == 200:
            data = response.json()
            print(data)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            
    def link_transfer(self, name, doctype, links: dict):
        data = {
            f"customer_primary_{doctype.lower()}": name,  # ID del contacto como primario
        }

        for link in links:
            endpoint = f"/api/resource/{link['link_doctype']}/{link['link_name']}"
            response = requests.put(f"{self.base_url}{endpoint}", json=data, headers=self.headers)
        
        endpoint_link = f"/api/resource/{doctype}/{name}"
        data = {"links": links}
        response_link = requests.put(f"{self.base_url}{endpoint_link}", json=data, headers=self.headers)
        if response_link.status_code == 200 and response.status_code == 200:
            print(f"{link['link_name']} actualizado exitosamente")
    
    def create(self, doctype: str, data: dict, links: dict = None) -> str:
        endpoint = f'/api/resource/{doctype}'

        response = requests.post(f"{self.base_url}{endpoint}", json=data, headers=self.headers)

        name = None
        
        if links: data['links'] = links

        if response.status_code == 200:
            print(f"{doctype} creado exitosamente:")
            resp = response.json()
            print(resp)
            name = resp['data']['name']
            if links: self.link_transfer(name, doctype, data['links'])
        else:
            print(f"Error al crear el {doctype}: {response.status_code} - {response.text}")
            
        
        return json.loads(response.text)['data']
    
    def create_mapping_cf(self, mappings) -> Contact:       

        if len(mappings) != 12:
            raise ValueError("El tamaño de 'mappings' no coincide con el número de campos en Contact.")
                
        return Contact(*mappings)


    def create_salesorder(self, customer: list, dependents: list):
        endpoint = f'/api/resource/Sales Order'
        
        contact = customer['contact_data'] if 'contact_data' in customer else customer
        
        if 'middlename' in customer and len(contact['middlename']) > 1:
            if 'firstname' in customer:
                name = f"{contact['firstname']} {contact['middlename']} {contact['lastname']}"
            else:
                name = f"{contact['first_name']} {contact['middle_name']} {contact['last_name']}"
        else:
            if 'firstname' in customer:
                name = f"{contact['firstname']} {contact['lastname']}"
            else:
                name = f"{contact['first_name']} {contact['last_name']}"
        
        data = {
            "customer": name, 
            "delivery_date": "2025-01-01",
            "custom_dependents": [
                {
                    "contact": name,
                    "relationship": contact['relationship'],
                    "smoke": contact['smoke'],
                    "pregnant": contact['pregnant'],
                    "apply": contact['apply'],
                }
            ] + [
                {
                    "contact": dependent['name'],
                    "relationship": dependent['relationship'],
                    "smoke": dependent['smoke'],
                    "pregnant": dependent['pregnant'],
                    "apply": dependent['apply'],
                } 
                for dependent in dependents
            ],
            "items": [
                {
                    "item_code": "BS",  # Reemplaza por el código del artículo
                    "qty": 1,
                    "rate": 100.0  # Precio del artículo
                }
            ],
            "currency": "USD",  # Moneda (puede cambiarse según la configuración)
            "selling_price_list": "Standard Selling",  # Lista de precios
        }

        response = requests.post(f"{self.base_url}{endpoint}", json=data, headers=self.headers)

        if response.status_code == 200:
            print("Sales Order creada exitosamente:", response.json())
        else:
            print("Error al crear la Sales Order:", response.status_code, response.text)
    
    def get_id(self, name):
        endpoint = f'/api/resource/Customer'
        
        filters = [
            ["name", "=", name]
        ]

        response = requests.get(f"{self.base_url}{endpoint}", params={'filters': json.dumps(filters)}, headers=self.headers)

        if response.status_code == 200:
            resp = response.json()
            if len(resp['data']) > 0:
                return False
            else:
                print(f"No existe nadie llamado: {name}")
                return True 
        else:
            print(f"Error al filtrar: {filters}")
            return False
    
    def create_bank_info(self, salesorder_data: list):
        endpoint = f'/api/resource/Bank Card'
        
        if len(salesorder_data['cf_1465']) > 1:
            card_types = {
                "Débito": "Debit",
                "Crédito": "Credit",
            }
            
            data = {
                "card_number": salesorder_data['cf_1465'], 
                "card_type": card_types[salesorder_data['cf_2157']],
                "expiration": salesorder_data['cf_1473'],
                "cvc": salesorder_data['cf_1477'],
            }

            response = requests.post(f"{self.base_url}{endpoint}", json=data, headers=self.headers)

            if response.status_code == 200:
                print("Sales Order creada exitosamente:", response.json())
            else:
                print("Error al crear la Sales Order:", response.status_code, response.text)
    
        if len(["salesorder_data['cf_1479']"]) > 1:
            banks = {
                "JPMorgan Chase Bank",
                "Bank Of America",
                "Wells Fargo Bank",
                "Citibank",
                "U.S. Bank",
                "PNC Bank",
                "Truist Bank",
                "Capital One",
                "Goldman Sachs Bank"
                "SOFI Bank",
                "TD Bank",
                "BMO Bank",
                "Citizens Bank",
                "First Citizens Bank",
                "M&T Bank",
                "Fifth Third Bank",
                "Huntington National Bank",
                "American Express National Bank",
                "Key Bank",
                "Ally Bank",
                "HSBC Bank USA",
                "Forbright Bank",
                "Barclays"
            }
            
            bank_types = {
                "Saving",
                "Current"
            }
            
            bank_subtypes = {
                "Personal",
                "Business"
            }
            
            data = {
                "account_name": salesorder_data['cf_1465'],
                "bank": banks,
                "account_type": bank_types,
                "account_subtype": bank_subtypes,
                "party_type": "",
                "party": "",
            }
            
            response = requests.post(f"{self.base_url}{endpoint}", json=data, headers=self.headers)

            if response.status_code == 200:
                print("Sales Order creada exitosamente:", response.json())
            else:
                print("Error al crear la Sales Order:", response.status_code, response.text)
            

In [14]:
from api.sync import Sync

class ERPMapping:
    def __init__(self, host, api_key, api_secret):
        self.erps = ERPSync(host, api_key, api_secret)
        
    def create_owner(self, salesorder_data, contact):    
        if owner_data := self.create_owner_data(salesorder_data, contact):
            try:
                customer = self.erps.create('Customer', owner_data['customer_data'])
                customer_links = [
                    {
                        "link_doctype": "Customer",
                        "link_name": customer['name']
                    }
                ]
                self.erps.create('Address', owner_data['address_data'], customer_links)
                self.erps.create('Contact', owner_data['contact_data'], customer_links)
                return {**owner_data['contact_data'], **owner_data['additional_data']}
            except:
                return owner_data
    
    def create_dependent_data(self, salesorder_data):
        mappings_cf = [
            ["spouse","cf_2347","cf_2349","cf_2351","cf_2357","cf_2355","cf_2359","cf_2389","cf_2709","cf_2383","","cf_2603",],
            ["dependent_1","cf_2405","cf_2407","cf_2409","cf_2417","cf_2413","cf_2411","cf_2401","cf_2711","cf_2423","cf_2431","cf_2605"],
            ["dependent_2","cf_2443","cf_2445","cf_2447","cf_2455","cf_2451","cf_2449","cf_2439","cf_2713","cf_2459","cf_2465","cf_2607"],
            ["dependent_3","cf_2479","cf_2481","cf_2483","cf_2491","cf_2487","cf_2485","cf_2475","cf_2715","cf_2495","cf_2501","cf_2609"],
            ["dependent_4","cf_2515","cf_2517","cf_2519","cf_2527","cf_2523","cf_2521","cf_2511","cf_2717","cf_2531","cf_2537","cf_2639"],
            ["dependent_5","cf_2645","cf_2647","cf_2649","cf_2687","cf_2681","cf_2679","cf_2615","cf_2719","cf_2685","cf_2699","cf_2683"],
        ]

        dependent_data = []  
        
        for dependent_cf in mappings_cf:
            dependent: Contact = self.erps.create_mapping_cf(dependent_cf)
            
            if salesorder_data[dependent.apply] != '':
                dependent_data.append(
                    {
                        
                        "first_name": salesorder_data[dependent.firstname],
                        "middle_name": salesorder_data[dependent.middlename],
                        "last_name": salesorder_data[dependent.lastname],
                        "custom_ssn": salesorder_data[dependent.ssn],
                        "custom_day_of_birth": salesorder_data[dependent.dob],
                        "gender": salesorder_data[dependent.gender],
                        "custom_document": documents[salesorder_data[dependent.document]] if salesorder_data[dependent.document] != '' else '',
                        "apply": True if salesorder_data[dependent.apply] == "OBAMACARE" else False,
                        "smoke": False if salesorder_data[dependent.smoke] == "NOT" else True,
                        "pregnant": True if salesorder_data[dependent.pregnant] == "NOT" else False,
                        "relationship": 'Spouse' if dependent.type == 'spouse' else (relationships[salesorder_data[dependent.relationship]] if salesorder_data[dependent.relationship] != '' else ''),
                    },
                )
        
        return dependent_data
    
    def create_dependent(self, salesorder_data):
        dependent_data = self.create_dependent_data(salesorder_data)
        dependents = []
        for data in dependent_data:
            new_contact = self.erps.create('Contact', data)
            other_info = {
                "relationship": data['relationship'],
                "smoke": data['smoke'],
                "apply": data['apply'],
                "pregnant": data['pregnant']
            }
            contact = {**new_contact, **other_info}
            dependents.append(contact)

        return dependents

    def create_owner_data(self, salesorder_data, contact):
        
        owner_data = []
        
        if contact['middlename'] and len(contact['middlename']) > 1:
            name = f"{contact['firstname']} {contact['middlename']} {contact['lastname']}"
        else:
            name = f"{contact['firstname']} {contact['lastname']}"
            
        documents = {
            'CIUDADANO': 'Passport',
            'ANTORCHA': 'Resident (I-551)',
            'RESIDENTE': 'Resident (I-551)',
            'ASILO POLITICO': 'Notice of Action (I-797)',
            'PASAPORTE DE VIAJERO': 'Refugee Travel (I-571)',
            'PERMISO DE TRABAJO': 'Employment Authorization (I-766)',
            'VISA': 'Visa (temporary I-551)',
            'NOTICIA DE ACCION': 'Notice of Action (I-797)',
            'I-94': 'Alien number or I-94 number',
        }
        
        if self.erps.get_id(name):
            owner_data = {
                "contact_data": {
                    "first_name": contact['firstname'],
                    "middle_name": contact['middlename'],
                    "last_name": contact['lastname'],
                    "custom_day_of_birth": contact['dob'],
                    "custom_ssn": ''.join([char for char in contact['ssn'] if char.isdigit()]),
                    "gender": str(contact['gender']).lower().capitalize(),
                    "email_ids": [
                        {
                            "email_id": contact['email'] if contact['email'] != '' else "test@yopmail.com",  
                            "is_primary": 1,
                        }
                    ],
                    "phone_nos": [
                        {
                            "phone": contact['phone_1'],
                            "is_primary": 1,
                        }
                    ]
                },
                "customer_data": {
                    "customer_name": name,
                    "customer_type": "Individual",
                    "territory": "All Territories"
                },
                "address_data": {
                    "address_title": name,
                    "address_line1": salesorder_data['cf_2737'],
                    "city": salesorder_data['cf_2739'],
                    "state": salesorder_data['cf_2765'],
                    "pincode": salesorder_data['cf_2743'],
                    "country": "United States",
                },
                "additional_data": {
                    "custom_document": documents[contact['document']] if contact['document'] != '' else '',
                    "apply": contact['apply'],
                    "smoke": contact['smoke'],
                    "pregnant": contact['pregnant'],
                    "relationship": 'Owner',
                },
            }
            print("Retorno owner_data")
            return owner_data
        else: 
            print("Retorno contact")
            return contact
      
    def export(self, salesorder_data, contact):
        owner = self.create_owner(salesorder_data, contact)
        dependents = self.create_dependent(salesorder_data)
        salesorder_name = self.create_salesorder(owner, dependents)
        

In [15]:
module = "SalesOrder"
condition = {
    "cf_2059" :"2025-01-01"
}

vts = Sync('http://192.168.99.102/vtigercrm_2022', 'superadmin', 'MFaeyxCMTmRrUZiE')
module_object = vts.search(module, condition)

service_id = "24x3"
product_id = "14x11"
assigned_user_id = "19x1"


19x113


In [32]:
salesorder = module_object[45]


contact = vts.get_contact_from_id(salesorder['contact_id'])

In [33]:
erpm = ERPMapping('http://development.localhost:8000', '76147fbeaf0fcf0', 'a418e14ab3bc4e9')
erpm.export(salesorder, contact)

{'message': 'Administrator'}
No existe nadie llamado: OSCAR KUS
Retorno owner_data
Customer creado exitosamente:
{'data': {'name': 'OSCAR KUS', 'owner': 'Administrator', 'creation': '2024-10-14 08:59:10.026917', 'modified': '2024-10-14 08:59:10.026917', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'naming_series': 'CUST-.YYYY.-', 'customer_name': 'OSCAR KUS', 'customer_type': 'Individual', 'territory': 'All Territories', 'is_internal_customer': 0, 'language': 'en', 'default_commission_rate': 0.0, 'so_required': 0, 'dn_required': 0, 'exempt_from_sales_tax': 0, 'is_frozen': 0, 'disabled': 0, 'doctype': 'Customer', 'companies': [], 'credit_limits': [], 'accounts': [], 'portal_users': [], 'sales_team': []}}
Address creado exitosamente:
{'data': {'name': 'OSCAR KUS-Billing', 'owner': 'Administrator', 'creation': '2024-10-14 08:59:10.123646', 'modified': '2024-10-14 08:59:10.123646', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'address_title': 'OSCAR KUS', 'address